# Setup Cloud SQL - Python Connector
---

Install [cloud-sql-python connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector#how-to-use-this-connector) using [pymysql](https://github.com/PyMySQL/PyMySQL) as the driver

In [ ]:
!pip install cloud-sql-python-connector[pymysql]

Authenticate using ***application default credentials***

In [ ]:
!gcloud auth application-default login

### Code

In [ ]:
# import connector
from google.cloud.sql.connector import connector, IPTypes

# import pandas
import pandas as pd

In [ ]:
conn = connector.connect(
    "dummy-329203:us-central1:dummy-mysql",
    "pymysql",
    ip_types=IPTypes.PUBLIC,
    user="root",
    password="CKnEx4mOtx2EADil",
    db="mysql"
)

In [ ]:
cursor = conn.cursor()
cursor.execute("SHOW COLUMNS FROM sys.sys_config")

columns = cursor.fetchall()
column_names = [col[0] for col in columns]

cursor.execute("SELECT * from sys.sys_config")
data = cursor.fetchall()

df = pd.DataFrame(data, columns=column_names)
df

,variable,value,set_time,set_by
0,diagnostics.allow_i_s_tables,OFF,2021-10-23 11:44:44,None
1,diagnostics.include_raw,OFF,2021-10-23 11:44:44,None
2,ps_thread_trx_info.max_length,65535,2021-10-23 11:44:44,None
3,statement_performance_analyzer.limit,100,2021-10-23 11:44:44,None
4,statement_performance_analyzer.view,None,2021-10-23 11:44:44,None
5,statement_truncate_len,64,2021-10-23 11:44:44,None


# Setup Cloud SQL - SQL Alchemy
---

Install [pymysql](https://docs.sqlalchemy.org/en/14/dialects/mysql.html#module-sqlalchemy.dialects.mysql.pymysql)

In [5]:
!pip install pymysql

     |████████████████████████████████| 43 kB 1.8 MB/s 


Authenticate using ***application default credentials***

In [ ]:
!gcloud auth application-default login

Need to add new network if use public IP: `0.0.0.0/0`



### Code

In [2]:
# import create_engine from sql alchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

# import pandas
import pandas as pd

# import drive
from google.colab import drive

Mount gdrive folder to get SSL files

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


Build connection without SSL

In [ ]:
engine = create_engine(
  URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="CKnEx4mOtx2EADil",
    host="34.72.191.193",
    port=3306,
    database="mysql"
  )
)

conn = engine.connect()

Build connection with SSL

In [6]:
path_to_ssl = '/content/drive/MyDrive/Colab Notebooks/creds/SSL'

ssl_files={
    "ssl_ca": f"{path_to_ssl}/server-ca.pem",
    "ssl_cert": f"{path_to_ssl}/client-cert.pem",
    "ssl_key": f"{path_to_ssl}/client-key.pem"
}

engine = create_engine(
  URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="CKnEx4mOtx2EADil",
    host="34.72.191.193",
    port=3306,
    database="mysql"
  ),
  connect_args=ssl_files
)

conn = engine.connect()

In [7]:
query_get_cols = "SHOW COLUMNS FROM sys.sys_config"
query_result = conn.execute(query_get_cols)
column_names = query_result.fetchall()
columns = [col[0] for col in column_names]

query_get_data = "select * from sys.sys_config"
query_result = conn.execute(query_get_data)
data = query_result.fetchall()

df = pd.DataFrame(data, columns=columns)
df

,variable,value,set_time,set_by
0,diagnostics.allow_i_s_tables,OFF,2021-10-23 11:44:44,None
1,diagnostics.include_raw,OFF,2021-10-23 11:44:44,None
2,ps_thread_trx_info.max_length,65535,2021-10-23 11:44:44,None
3,statement_performance_analyzer.limit,100,2021-10-23 11:44:44,None
4,statement_performance_analyzer.view,None,2021-10-23 11:44:44,None
5,statement_truncate_len,64,2021-10-23 11:44:44,None
